## Introduction to MacroMax

This is a simple example of how to use our open-source electromagnetics solver, MacroMax.

To take a full advatage of the available hardware on Google Colab, make sure to enable GPU in the setting of the notebook. In the menu bar above, select **Runtime > Change runtime type > Hardware accelerator > GPU**.[link text](https://)

All source code and further examples are available on [GitHub](https://github.com/tttom/MacroMax). The latest documentation of the library can be found [here](https://macromax.readthedocs.io/en/latest/).

Feel free contact me with questions or discussion at t.vettenburg@dundee.ac.uk.


## Install and load dependencies. Copy this at the start of your Colab script.

In [ ]:
# Capture disables display of output for this cell.
%%capture

!pip install -U macromax  # Install MacroMax. On your own computer, this can be done in a shell without the exclamation mark.

import macromax  # Load the main library
from macromax.bound import LinearBound  # We will also use an absorbing boundary in the example below
import numpy as np  # Load the standard NumPy library
import matplotlib.pyplot as plt  # Load the standard plotting library

## Define a scattering system, e.g. a glass cylinder.

The scattering system can be specified as a 1D, 2D, or 3D array, where each element represents the refractive index on a regular Cartesian grid. Here, we define the refractive index in code; however, it could just as easily be loaded from an image file.

MacroMax provides several tools to help define scattering systems. We will use:

1.   **Grid(shape, step)** to represent the Cartesian coordinate system and sample locations. This enables us to work with physical units, e.g. in meters, instead of pixels.
2.   **LinearBound(grid, thickness)** represents an absorbring boundary with an absorption coefficient that increases linearly towards the edge of the calculation volume. The default boundary condition is periodic.



In [ ]:
print(f'Using MacroMax version {macromax.__version__}.')

wavelength = 500e-9  # everything is in meters

# Define a Cartesian sample grid with a physical extent of 128/6 x 256/6 wavelengths
grid = macromax.Grid(shape=[128, 256], step=wavelength/6)  # grid[0] = vertical pos., grid[1] = horizontal pos.
# Define simple absorbing boundaries with extinction coefficient going from 0 to 0.1
bound = LinearBound(grid, thickness=5 * wavelength, max_extinction_coefficient=0.1)

# Define the scattering system
sphere_r = 5 * wavelength
sphere_offset_in_meters = 10 * wavelength
refractive_index = 1 + 0.5 * (np.sqrt(grid[0] ** 2 + (grid[1] - sphere_offset_in_meters) ** 2) < sphere_r)  # Glass sphere on the right-hand side

# Display the refractive index distribution [1, 1.5]
im = plt.imshow(refractive_index)
plt.colorbar(im)
plt.show()

## Next, we specify a source of radiation and compute its field.

In this example we define a scalar point source. More generally, any complex distribution on the Cartesian grid can be used, e.g. to model incident plane waves. Polarization can be specified by adding a dimension of length 3 on the right.

The field produced by the source can be calculated by the **macromax.solve()** function. In this particular example, it is scalar.

In [ ]:
# Define the source on the same grid
source = np.zeros(grid.shape)  # Our source is zero everywhere but in the next point
source[grid.shape[0] // 2, grid.shape[1] // 4] = 1  # A plane wave source could be defined by a complex-valued distribution on one side of the calculation volume.

# The actual calculation is done here!
solution = macromax.solve(grid, vacuum_wavelength=wavelength, bound=bound,  # boundaries of 0 thickness are interpreted as periodic.
                          source_distribution=source,  # Alternatively, a (vectorial) current density distribution can be specified.
                          refractive_index=refractive_index)  # Alternatively, permittivity, permeability, etc. can be specified.

field = solution.E[0]  # Only the first polarization component is meaningful for this scalar example.

# Displaying the real part of the field
fig, axs = plt.subplots(dpi=150)
axs.imshow(field.real)
axs.set_title(r"$\Re (E)$")
plt.show()

### Macromax also includes some tools to visualize complex fields in physical coordinates.

In [ ]:
from macromax.utils.display import complex2rgb, grid2extent

fig, axs = plt.subplots(dpi=150)
axs.imshow(complex2rgb(field, normalization=3), extent=grid2extent(grid) / 1e-6)  # Display the field amplitude as brightness and its phase as color hue.
axs.set(xlabel=r'x [$\mu m$]', ylabel=r'y [$\mu m$]', title=r'scattered field')
plt.show()